<a href="https://colab.research.google.com/github/DininduChamikara/Research_NLP/blob/main/Sinhala_NER_2_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install simpletransformers

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from simpletransformers.ner import NERModel,NERArgs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 20.6 MB/s eta 0:00:00

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Research Project/ner_dataset_v3.0.csv', encoding = "latin1")

In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)
data["labels"] = data["labels"].str.upper()

In [ ]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [ ]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [ ]:
label = data["labels"].unique().tolist()

print(label)

['LOCATION', 'OTHER', 'PERSON', 'DATE', 'ORGANIZATION', 'TIME']


In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda= False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train_model(train_data,eval_data=test_data)

In [11]:
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PERSON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOCATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORGANIZATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_

In [12]:
result

{'eval_loss': 0.5269305450575692,
 'precision': 0.8888888888888888,
 'recall': 0.021220159151193633,
 'f1_score': 0.04145077720207253}

In [13]:
prediction, model_output = model.predict(["ආසියානු ක්‍රිකට් ශුර ශ්‍රී ලංකා කණ්ඩායම සහ සත්කාරක ඉන්දීය කණ්ඩායම"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
prediction

[[{'ආසියානු': 'OTHER'},
  {'ක්\u200dරිකට්': 'OTHER'},
  {'ශුර': 'OTHER'},
  {'ශ්\u200dරී': 'OTHER'},
  {'ලංකා': 'OTHER'},
  {'කණ්ඩායම': 'OTHER'},
  {'සහ': 'OTHER'},
  {'සත්කාරක': 'OTHER'},
  {'ඉන්දීය': 'OTHER'},
  {'කණ්ඩායම': 'OTHER'}]]

In [16]:
prediction, model_output = model.predict(["පිළිතුරු ඉනිම ක්‍රීඩා කළ ශ්‍රී ලංකා කණ්ඩායමේ ආරම්භක පිතිකරුවන් වන පැතුම් නිශ්ශංක සහ කුසල් මෙන්ඩිස් තරගයට මනා ආරම්භයක් ලබා දීමට උත්සහ කළත් ඔවුන්ට සාර්ථක වීම ඉන්දීය පන්දු යවන්නන්ගේ බල ඇණියෙන් ඉඩක් නොවීය."])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
prediction

[[{'පිළිතුරු': 'OTHER'},
  {'ඉනිම': 'OTHER'},
  {'ක්\u200dරීඩා': 'OTHER'},
  {'කළ': 'OTHER'},
  {'ශ්\u200dරී': 'OTHER'},
  {'ලංකා': 'OTHER'},
  {'කණ්ඩායමේ': 'OTHER'},
  {'ආරම්භක': 'OTHER'},
  {'පිතිකරුවන්': 'OTHER'},
  {'වන': 'OTHER'},
  {'පැතුම්': 'OTHER'},
  {'නිශ්ශංක': 'OTHER'},
  {'සහ': 'OTHER'},
  {'කුසල්': 'OTHER'},
  {'මෙන්ඩිස්': 'OTHER'},
  {'තරගයට': 'OTHER'},
  {'මනා': 'OTHER'},
  {'ආරම්භයක්': 'OTHER'},
  {'ලබා': 'OTHER'},
  {'දීමට': 'OTHER'},
  {'උත්සහ': 'OTHER'},
  {'කළත්': 'OTHER'},
  {'ඔවුන්ට': 'OTHER'},
  {'සාර්ථක': 'OTHER'},
  {'වීම': 'OTHER'},
  {'ඉන්දීය': 'OTHER'},
  {'පන්දු': 'OTHER'},
  {'යවන්නන්ගේ': 'OTHER'},
  {'බල': 'OTHER'},
  {'ඇණියෙන්': 'OTHER'},
  {'ඉඩක්': 'OTHER'},
  {'නොවීය.': 'OTHER'}]]